In [1]:
import requests
from bs4 import BeautifulSoup as bs
import yaml
from IPython.display import display
from time import sleep
import re
from tqdm import tqdm
import pandas as pd




In [2]:
from typing import List, Dict, Any, Set, Tuple, Union, Optional

In [3]:
url = "http://factcheck.snu.ac.kr/v2/facts/%d"

In [4]:
class Speaking:
    def __init__(self, num: int, predata: Optional[Dict[Any, Any]] = None):
        self.num = num
        if predata is None:
            self.responce = requests.get(url % self.num)
            if self.responce.status_code != 200:
                raise Exception(f"{self.responce.status_code} error at page {self.num}.")
            self.soup = bs(self.responce.text, 'html.parser')\
                # bs를 이용하여 구문 분석 후 soup에 저장합니다.
            self.detail = self.soup.select_one(".fcItem_detail_top") # 상세 페이지를 추출합니다.
            self.speacker = self.detail.select_one(".name").text.strip() # 발언자를 추출합니다.
            self.title = self.detail.select_one(".fcItem_detail_li_p > p:nth-child(1) > a").text.strip() # 제목을 추출합니다.
            self.source_element = self.detail.select_one(".source") # 출처가 담긴 html 요소를 추출합니다.
            self.source = {self.source_element.text.strip(): ""} \
                if self.source_element.select_one("a") == None\
                else {self.source_element.select_one("a").text.strip(): self.source_element.select_one("a")['href']} # 출처를 추출합니다.
            self.cartegories = {li.text for li in self.detail.select(".fcItem_detail_bottom li")} # 카테고리를 추출합니다.
            self.explain = self.soup.select_one(".exp").text.strip() # 설명을 추출합니다.
            self.factchecks = self.get_fc(self.soup)
        else:
            self.speacker = predata['speacker']
            self.title = predata['title']
            self.source = predata['source']
            self.cartegories = predata['cartegories']
            self.explain = predata['explain']
            self.factchecks = predata['factchecks']
    
    def as_dict(self): # 데이터를 딕셔너리 형태로 반환합니다.
        return {
            'speacker': self.speacker,
            'title': self.title,
            'source': self.source,
            'cartegories': self.cartegories,
            'explain': self.explain,
            'factchecks': self.factchecks
        }
    
    def as_yaml(self): # 데이터를 yaml 형태로 반환합니다.
        return yaml.dump(self.as_dict(), allow_unicode=True)

    def save_as_yaml(self, path:str):
        with open(path, 'w') as f:
            f.write(self.as_yaml())

    # SNU 팩트체크 사이트는 다음과 같이 팩트 체크 별 아이디와 점수를 보냅니다.
    # <ul> <li class="fcItem_vf_li">...</li>
    # <script charset="utf-8" type="text/javascript">
    # $(function () { showScore(아이디, 점수)}); </script> ... </ul>
    # 따라서 아이디와 점수를 추출하는 정규표현식을 사용합니다.

    
    # 아이디와 점수를 추출하는 함수를 정의합니다.

    def get_fc_id_score(self, soup:bs) -> List[Tuple[int, int]]:
        id_score: List[Tuple[int,int]] = [] # 아이디와 점수를 저장할 리스트
        id_score_re = re.compile(r'(?<=showScore\()\d+, \d+')
        for fc_item_script in soup.select(".fcItem_vf > ul > script"): # 팩트 체크 아이템의 스크립트를 찾습니다.
            id_score_str: str = id_score_re.search(fc_item_script.text).group() # 정규 표현식으로 아이디와 점수값을 담은 문자열을 추출합니다.
            id_score_int: Tuple = tuple(map(int, id_score_str.split(', '))) # 문자열을 정수로 변환합니다.
            id_score.append(id_score_int)
        return id_score



    # 작성 시간과 내용을 추출하는 함수를 정의합니다.

    def get_fc_time_contents(self, soup:bs) -> List[Tuple[str, str, str]]:
        time_contents: List[str] = [] # 시간과 내용을 저장할 리스트
        for fc_item in soup.select(".fcItem_vf > ul > li"): # 팩트 체크 아이템을 찾습니다.
            date = fc_item.select_one(".reg_date > p i:nth-child(1)").text # 작성 날짜를 추출합니다.
            time = fc_item.select_one(".reg_date > p i:nth-child(2)").text # 작성 시간을 추출합니다.
            raw_content = fc_item.select_one(".vf_exp_wrap").text # 팩트 체크 내용을 추출합니다.
            content = re.sub(r'\s{2,}', " ", raw_content.strip()) # 공백을 제거합니다.
            time_contents.append((date, time, content))
        return time_contents

    # 위의 함수들의 반환을 합쳐 최종적으로 아이디를 키로 갖고 나머지 내용을 값으로 갖는 딕셔너리를 반환하는 함수를 정의합니다.

    def get_fc(self, soup:bs) -> Dict[int, Dict[Any, Any]]:
        id_score = self.get_fc_id_score(soup) # 아이디와 점수를 추출합니다.
        time_contents = self.get_fc_time_contents(soup) # 작성 시간과 내용을 추출합니다.
        zipped = zip(id_score, time_contents) # 두 리스트를 합쳐 쌍을 생성합니다.
        return {id: {'score': score, 'date': date, 'time': time, 'content': content} for (id, score), (date, time, content) in zipped}


In [ ]:
# 더이상 페이지가 없을 때까지 페이지들을 수집합니다.
speakings = []
errors = set()
for i in tqdm(range(1,5000)):
    try:
        speakings.append(Speaking(i))
    except:
        speakings.append(None)
        errors.add(i)
        if i > 3670:
            break
    if i % 10 == 0:
        sleep(1)

In [19]:
speaks_dict = {speaking.num: speaking.as_dict() for speaking in speakings if speaking is not None}
yaml.dump(speaks_dict, open('speakings.yaml', 'w', encoding="utf-8"), default_flow_style=False, allow_unicode=True)

In [6]:
speaks_dict = yaml.load(open('speakings.yaml', 'r', encoding="utf-8"), Loader=yaml.FullLoader)
speakings = [Speaking(id, contents) for id, contents in speaks_dict.items()]
for id in range(10):
    print(speakings[id].as_dict()['title'])

현재 지지율 1위 문재인 민주당 경선 후보 아들과 관련한 취업특혜 논란이 있다.
“77%로 미군 주둔비 부담 커…일본은 50%”
"더불어민주당도 국모닝 했다"
"대통령선거 본선에 나가기 직전에 사표를 제출하면 보궐선거는 없다"
“공공부문 일자리 81만개 만들 수 있다”
문재인캠프 영입인사의 과거 이력에 관한 주장은 사실과 다르다.
세월호 희생자들이 천안함 희생자들보다 보상을 많이 받았다는 주장과 세월호 관련 법안이 국회서 어떻게 처리되고 있는지 검증
Q: 자유한국당 홍준표 경남지사가 위안부 합의에 대해 "대통령이 되면 합의를 파기할 것"이라고 했습니다.
"홍준표 지사가 국회 운영위원장 시절, 판공비 일부를 집에서 쓰게 한 것은 문제다."

"민주당의 호남권 대선후보 경선 결과 무효표가 무려 10만여 표가 나왔다."


In [14]:
import classes
classes.load_speakings()[0].as_dict()

{'speacker': '문재인',
 'title': '현재 지지율 1위 문재인 민주당 경선 후보 아들과 관련한 취업특혜 논란이 있다.',
 'source': {'[사실은] 문재인 아들 취업 특혜 논란…감사 내용 보니': 'http://news.sbs.co.kr/news/endPage.do?news_id=N1004112510&plink=TITLE&cooper=SBSNEWSINFACT'},
 'cartegories': {'정치, 19대 대선, 대통령 선거', '정치인(공직자)과 관련된 사실'},
 'explain': '현재 지지율 1위 문재인 민주당 경선 후보 아들의 취업특혜 논란과 관련된 가짜 뉴스는 무엇이며 남은 의혹은 무엇인지를 취재.',
 'factchecks': {1: {'content': "검증내용 <앵커>대선 국면에 접어들면서 저희 사실은 코너에선 선거 관련 가짜 뉴스와 대선 후보 관련 논란을 집중 검증하고 있습니다. 마침 저희 사실은 홈페이지에 지지율 1위 문재인 민주당 후보의 아들 취업 특혜 논란을 따져봐 달라는 요청이 들어와서, 오늘(25일)은 그 문제를 한번 집중적으로 따져 보겠습니다.박세용 기자, 먼저 논란의 핵심부터 정리해주시죠.<기자>문재인 후보 아들이 노무현 정부 시절인 2007년 1월에 한국고용정보원이라는 공기업에 합격합니다.당시 고용정보원의 원장이 청와대에서 문재인 후보와 같이 근무했던 사람이거든요.그래서 아버지 덕에 취업한 거 아니냐는 의혹이\xa0나오는 겁니다.<앵커>이런 의혹이 자꾸 나오니까 선관위는 최근 이런 의혹들은 모두 허위라고 하면서 관련 게시물을 단속하고 있잖아요?<기자>선관위가 지금 단속하는 건 악의적인 가짜 뉴스입니다.문 후보 아들 1명만 지원해서 혼자 합격했다, 또 5급 공무원 자리다, 이런 글들이\xa0돌고 있는데\xa0사실이 아니라 거짓이라는 겁니다.실제로는 2명 지원해서 2명 합격했고, 공무원이 아니라 공기업 일반직입니다.<앵커>이런 의혹에 대해서 문재인 후보 측은 2007년과 2010년 이미 두 번 다 감